In [ ]:
import os

def delete_temp_files(file_name):
    temp_folder = os.path.join(os.getcwd(), 'temps')
    file_path = os.path.join(temp_folder, file_name)
    
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Deleted temporary file: {file_path}")
    else:
        print(f"File not found: {file_path}")

In [ ]:
import os
import uuid

from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

load_dotenv()

def upload_pdf_to_blob(pdf_bytes: bytes) -> str:
    container_name = os.getenv("AZURE_BLOB_CONTAINER_NAME")
    connection_string = os.getenv("AZURE_BLOB_CONNECTION_STRING")

    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=f"{str(uuid.uuid4())}.pdf")
    blob_client.upload_blob(pdf_bytes, overwrite=True)
    
    return blob_client.url

In [ ]:
from PyPDF2 import PdfReader, PdfWriter

def protect_pdf(password: str, file_name: str):
    # 1. Load your existing PDF
    reader = PdfReader(f"temps/{file_name}.pdf")

    # 2. Copy pages into a new writer
    writer = PdfWriter()
    for page in reader.pages:
        writer.add_page(page)

    # 3. Encrypt with your chosen password
    writer.encrypt(user_password=password, owner_pwd=None, use_128bit=True)

    # 4. Write out the password-protected file. wb = "write binary" mode
    with open(f"temps/{file_name}-protected.pdf", "wb") as out_f:
        writer.write(out_f)
        
    print(f"Password-protected PDF saved as: temps/{file_name}-protected.pdf")

In [ ]:
import os
import requests

from docx2pdf import convert
from io import BytesIO

def convert_docx_to_pdf(remote_url: str, file_name: str):
    
    # Download DOCX to memory
    response = requests.get(remote_url)
    docx_bytes = BytesIO(response.content)
    
    # Ensure the temps directory exists
    os.makedirs("temps", exist_ok=True)
    
    # Save DOCX to a temporary file (docx2pdf requires a file path)
    with open(f"temps/{file_name}.docx", "wb") as temp_docx:
        temp_docx.write(docx_bytes.getbuffer())
        
    convert(f"temps/{file_name}.docx", f"temps/{file_name}.pdf")
    
    print(f"Converted {file_name}.docx to {file_name}.pdf in temps directory.")

In [ ]:
import uuid

remote_url = "[ADD YOUR DOCX URL HERE]"
file_name = str(uuid.uuid4())
password = "12345"

print(f"File name: {file_name}")

# Convert DOCX to PDF
convert_docx_to_pdf(remote_url, file_name)

# Protect the PDF with a password
protect_pdf(password, file_name)

# Upload the protected PDF to Azure Blob Storage
url = upload_pdf_to_blob(open(f"temps/{file_name}-protected.pdf", "rb").read())
print(f"Protected PDF URL: {url}")

# Clean up temporary files
delete_temp_files(f"{file_name}.docx")
delete_temp_files(f"{file_name}.pdf")
delete_temp_files(f"{file_name}-protected.pdf")